In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
from xgboost import XGBClassifier
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

from tensorflow.keras import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

In [2]:
# read the data
data = pd.read_csv("combine_test_data.csv")
test_data = data.sample(frac=1).reset_index(drop=True)
# split the dataset and make the train and test data sets
X = test_data.drop(['label'],axis=1).values
y = test_data['label'].values
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=10)

## XGB Method

In [3]:
# make XGB model
xgb = xgb.XGBClassifier(objective='binary:logistic',
                                learning_rate=0.4,
                                max_depth=7,
                                )
#fit the model
X_train1 = X_train.copy()
X_test1 = X_test.copy()
y_train1 = y_train.copy()
y_test1 = y_test.copy()
xgb.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.4, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=7, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [4]:
#predicting the target value from the model for the samples
y_test_xgb = xgb.predict(X_test1)
y_train_xgb = xgb.predict(X_train1)

#computing the accuracy of the model performance
acc_train_xgb = accuracy_score(y_train1,y_train_xgb)
acc_test_xgb = accuracy_score(y_test1,y_test_xgb)

print("XGBoost: Accuracy on training Data: {:.3f}".format(acc_train_xgb))
print("XGBoost : Accuracy on test Data: {:.3f}".format(acc_test_xgb))

XGBoost: Accuracy on training Data: 0.969
XGBoost : Accuracy on test Data: 0.948


## Neural Network

In [5]:
# copy the data sets
X_train2 = X_train.copy()
X_test2 = X_test.copy()
y_train2 = y_train.copy()
y_test2 = y_test.copy()
# load and transform data
scaler = StandardScaler()
X_train2 = scaler.fit_transform(X_train2)
X_test2 = scaler.transform(X_test2)

In [6]:
nn_model = Sequential([
    Input(shape=(X_train2.shape[1],)), 
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid') 
])

nn_model.compile(optimizer='adam',loss='binary_crossentropy', metrics=['accuracy'])

history = nn_model.fit(X_train2, y_train2, epochs=15,validation_split=0.2)

Epoch 1/15
210/210 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8182 - loss: 0.4701 - val_accuracy: 0.8369 - val_loss: 0.3715
Epoch 2/15
210/210 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8671 - loss: 0.3161 - val_accuracy: 0.8423 - val_loss: 0.3419
Epoch 3/15
210/210 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8772 - loss: 0.2823 - val_accuracy: 0.8506 - val_loss: 0.3317
Epoch 4/15
210/210 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8788 - loss: 0.2752 - val_accuracy: 0.8565 - val_loss: 0.3158
Epoch 5/15
210/210 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8875 - loss: 0.2529 - val_accuracy: 0.8786 - val_loss: 0.3089
Epoch 6/15
210/210 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8936 - loss: 0.2472 - val_accuracy: 0.8702 - val_loss: 0.2921
Epoch 7/15
210/210 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8980 - loss: 0.2339 - val_accuracy: 0.8833 - val_loss: 0.2796
Epoch 8/15
210/210 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9060 - loss: 0.2236 - val_accuracy: 0.

In [7]:
test_loss, test_acc = nn_model.evaluate(X_test2, y_test2)
print(f"Test Accuracy: {test_acc}")

113/113 ━━━━━━━━━━━━━━━━━━━━ 0s 912us/step - accuracy: 0.9058 - loss: 0.2229
Test Accuracy: 0.9052777886390686
